In [3]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm

import pandas as pd

In [7]:
data = [] # creating list for data

browser = Chrome("./chromedriver") # opening Chrome with a special driver
url = "https://bus.gov.ru/registry" # link for web scraping
browser.get(url) # opening this link with Chrome using a driver

input_tab = browser.find_element(By.XPATH, '/html/body/div[2]/ui-view/form/div[1]/div[1]/label/input') 
# looking for input field
input_tab.send_keys("бассейн") # writing smth into the input field
button = browser.find_element(By.XPATH, '//button[@type="submit"]') # looking for "search" button
button.click() # clicking on the "search" button


for p in tqdm(range(4)): # series for the page switching
    
    sleep(7) # waiting for the page updating for 7 seconds
    
    soup = BeautifulSoup(browser.page_source, 'lxml') # saving a new page's code
    
    orgs = soup.findAll('div', class_ = 'result') # saving a code of elements with needed data 
    
    for org in orgs:
        # looking for an organisation's name and link in the saved page code
        try:
            name = org.find('a', class_ = 'result__title').text.strip()
        except:
            name = "-"
        try:
            link = "https://bus.gov.ru/registry" + soup.find('a', class_ = 'result__button_registry').get('href')
        except:
            link = "-"
        # adding new data to list
        data.append([name, link])
    
    
    try: 
        browser.find_element(By.CLASS_NAME, 'pagination__next').click() # clicking "next page" button
    except:
        continue
        
header = ["Name", "Link"]
df = pd.DataFrame(data, columns = header)
df.to_csv("./busgov.csv", sep = ";", encoding = "utf8")

<ipython-input-7-e09d6ed46bd9>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = Chrome("./chromedriver") # opening Chrome with a special driver
100%|██████████| 4/4 [00:28<00:00,  7.19s/it]
